In [7]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [42]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 1.x
except Exception:
    pass

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "tensorflow"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [24]:
import numpy as np
from sklearn.datasets import fetch_california_housing

reset_graph()

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

x = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='x')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
xt = tf.transpose(x)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(xt, x)), xt), y)

with tf.Session() as sess:
    theta_value = theta.eval()

In [25]:
theta_value

array([[-3.68901253e+01],
       [ 4.36643779e-01],
       [ 9.45042260e-03],
       [-1.07117996e-01],
       [ 6.43712580e-01],
       [-3.96291580e-06],
       [-3.78801115e-03],
       [-4.20931637e-01],
       [-4.34006572e-01]], dtype=float32)

In [60]:
# Batch Gradient Decent for parameter optimization with GradientDescentOptimizer

reset_graph()

n_epochs = 1000                                                                       # not shown in the book
learning_rate = 0.01                                                                  # not shown

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")            # not shown
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")            # not shown
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")                                      # not shown
error = y_pred - y                                                                    # not shown
mse = tf.reduce_mean(tf.square(error), name="mse")                                    # not shown
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)            # not shown
training_op = optimizer.minimize(mse)                                                 # not shown

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())                                # not shown
            save_path = saver.save(sess, "/tmp/my_model.ckpt")
        sess.run(training_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "/tmp/my_model_final.ckpt")

Epoch 0 MSE = 9.161542
INFO:tensorflow:/tmp\my_model.ckpt.data-00000-of-00001
INFO:tensorflow:0
INFO:tensorflow:/tmp\my_model.ckpt.index
INFO:tensorflow:0
INFO:tensorflow:/tmp\my_model.ckpt.meta
INFO:tensorflow:800
Epoch 100 MSE = 0.71450037
INFO:tensorflow:/tmp\my_model.ckpt.data-00000-of-00001
INFO:tensorflow:0
INFO:tensorflow:/tmp\my_model.ckpt.index
INFO:tensorflow:0
INFO:tensorflow:/tmp\my_model.ckpt.meta
INFO:tensorflow:800
Epoch 200 MSE = 0.5667048
INFO:tensorflow:/tmp\my_model.ckpt.data-00000-of-00001
INFO:tensorflow:0
INFO:tensorflow:/tmp\my_model.ckpt.index
INFO:tensorflow:0
INFO:tensorflow:/tmp\my_model.ckpt.meta
INFO:tensorflow:800
Epoch 300 MSE = 0.5555718
INFO:tensorflow:/tmp\my_model.ckpt.data-00000-of-00001
INFO:tensorflow:0
INFO:tensorflow:/tmp\my_model.ckpt.index
INFO:tensorflow:0
INFO:tensorflow:/tmp\my_model.ckpt.meta
INFO:tensorflow:800
Epoch 400 MSE = 0.54881126
INFO:tensorflow:/tmp\my_model.ckpt.data-00000-of-00001
INFO:tensorflow:0
INFO:tensorflow:/tmp\my_model.

In [61]:
with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")
    best_theta_restored = theta.eval()

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


In [62]:
np.allclose(best_theta, best_theta_restored)

True

In [63]:
reset_graph()
# notice that we start with an empty graph.

saver = tf.train.import_meta_graph("/tmp/my_model_final.ckpt.meta")  # this loads the graph structure
theta = tf.get_default_graph().get_tensor_by_name("theta:0") # not shown in the book

with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")  # this restores the graph's state
    best_theta_restored = theta.eval()

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


In [64]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [65]:
def make_log_subdir(run_id=None):
    if run_id is None:
        run_id = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    return "{}/run-{}/".format(root_logdir, run_id)

In [66]:
file_writer = tf.summary.FileWriter(logdir, graph=tf.get_default_graph())

In [67]:
file_writer.close()

In [68]:
%load_ext tensorboard

%tensorboard --logdir {root_logdir}

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 17548), started 0:54:50 ago. (Use '!kill 17548' to kill it.)

In [69]:
def save_graph(graph=None, run_id=None):
    if graph is None:
        graph = tf.get_default_graph()
    logdir = make_log_subdir(run_id)
    file_writer = tf.summary.FileWriter(logdir, graph=graph)
    file_writer.close()
    return logdir

In [70]:
save_graph()

'tf_logs/run-20220817213129/'

In [71]:

%tensorboard --logdir {root_logdir}

Reusing TensorBoard on port 6006 (pid 17548), started 0:56:02 ago. (Use '!kill 17548' to kill it.)